In [43]:
import time
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import csv


In [44]:
table = pd.read_csv("data/2021_22_matches", sep=",")
table.tail()

,id,season,matchday,date,time,home,guest,result,link
326,108859,2021_22,Spieltag 34,09.04.2022,19:30,TEAM EHINGEN URSPRING,RÖMERSTROM Gladiators Trier,69:101,http://live.2basketballbundesliga.de/g/108859?...
327,108855,2021_22,Spieltag 34,09.04.2022,19:30,Medipolis SC Jena,VfL SparkassenStars Bochum,121:83,http://live.2basketballbundesliga.de/g/108855?...
328,108860,2021_22,Spieltag 34,09.04.2022,19:30,VfL Kirchheim Knights,Nürnberg Falcons BC,57:89,http://live.2basketballbundesliga.de/g/108860?...
329,108857,2021_22,Spieltag 34,09.04.2022,19:30,Itzehoe Eagles,RASTA Vechta,67:84,http://live.2basketballbundesliga.de/g/108857?...
330,108854,2021_22,Spieltag 34,19.04.2022,18:00,ROSTOCK SEAWOLVES,PS Karlsruhe LIONS,73:80,http://live.2basketballbundesliga.de/g/108854?...


In [45]:
table['matchday'] = table['matchday'].str.replace("Spieltag ","")
table.tail()

,id,season,matchday,date,time,home,guest,result,link
326,108859,2021_22,34,09.04.2022,19:30,TEAM EHINGEN URSPRING,RÖMERSTROM Gladiators Trier,69:101,http://live.2basketballbundesliga.de/g/108859?...
327,108855,2021_22,34,09.04.2022,19:30,Medipolis SC Jena,VfL SparkassenStars Bochum,121:83,http://live.2basketballbundesliga.de/g/108855?...
328,108860,2021_22,34,09.04.2022,19:30,VfL Kirchheim Knights,Nürnberg Falcons BC,57:89,http://live.2basketballbundesliga.de/g/108860?...
329,108857,2021_22,34,09.04.2022,19:30,Itzehoe Eagles,RASTA Vechta,67:84,http://live.2basketballbundesliga.de/g/108857?...
330,108854,2021_22,34,19.04.2022,18:00,ROSTOCK SEAWOLVES,PS Karlsruhe LIONS,73:80,http://live.2basketballbundesliga.de/g/108854?...


In [40]:
table['matchday'].astype('str')
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331 entries, 0 to 330
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        331 non-null    int64 
 1   season    331 non-null    object
 2   matchday  331 non-null    object
 3   date      331 non-null    object
 4   time      297 non-null    object
 5   home      331 non-null    object
 6   guest     331 non-null    object
 7   result    331 non-null    object
 8   link      331 non-null    object
dtypes: int64(1), object(8)
memory usage: 23.4+ KB


In [41]:
table_new = table.drop(table.index[table['matchday'] == 'len(table["matchday"]) > 2'])
table_new.head()

,id,season,matchday,date,time,home,guest,result,link
0,108931,2021_22,PO-1/4 Finale Spiel 1,14.04.2022,19:30,Tigers Tübingen,Eisbären Bremerhaven,82:78,http://live.2basketballbundesliga.de/g/108931?...
1,108930,2021_22,PO-1/4 Finale Spiel 1,22.04.2022,19:30,RÖMERSTROM Gladiators Trier,Bayer Giants Leverkusen,92:68,http://live.2basketballbundesliga.de/g/108930?...
2,108933,2021_22,PO-1/4 Finale Spiel 1,22.04.2022,19:30,Medipolis SC Jena,Uni Baskets Paderborn,89:84,http://live.2basketballbundesliga.de/g/108933?...
3,108932,2021_22,PO-1/4 Finale Spiel 1,26.04.2022,19:30,ROSTOCK SEAWOLVES,PS Karlsruhe LIONS,107:103,http://live.2basketballbundesliga.de/g/108932?...
4,108936,2021_22,PO-1/4 Finale Spiel 2,17.04.2022,18:00,Eisbären Bremerhaven,Tigers Tübingen,75:82,http://live.2basketballbundesliga.de/g/108936?...


In [60]:
table_reg_season = table[table['matchday'].map(len) < 3]
table_reg_season.head()

,id,season,matchday,date,time,home,guest,result,link
25,108961,2021_22,1,N.N.,NaN,spielfrei,Itzehoe Eagles,--:--,http://live.2basketballbundesliga.de/g/108961?...
26,108556,2021_22,1,17.09.2021,19:30,RASTA Vechta,ROSTOCK SEAWOLVES,54:68,http://live.2basketballbundesliga.de/g/108556?...
27,108557,2021_22,1,18.09.2021,18:30,Nürnberg Falcons BC,Artland Dragons,74:71,http://live.2basketballbundesliga.de/g/108557?...
28,108558,2021_22,1,18.09.2021,19:00,VfL SparkassenStars Bochum,Medipolis SC Jena,74:91,http://live.2basketballbundesliga.de/g/108558?...
29,108559,2021_22,1,18.09.2021,19:00,Eisbären Bremerhaven,RÖMERSTROM Gladiators Trier,84:87,http://live.2basketballbundesliga.de/g/108559?...


In [61]:
table_reg_season.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306 entries, 25 to 330
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        306 non-null    int64 
 1   season    306 non-null    object
 2   matchday  306 non-null    object
 3   date      306 non-null    object
 4   time      272 non-null    object
 5   home      306 non-null    object
 6   guest     306 non-null    object
 7   result    306 non-null    object
 8   link      306 non-null    object
dtypes: int64(1), object(8)
memory usage: 23.9+ KB


In [69]:
table_reg_season = table_reg_season.drop('date', axis=1)
table_reg_season.head()

,season,matchday,home,guest,result
25,2021_22,1,spielfrei,Itzehoe Eagles,--:--
26,2021_22,1,RASTA Vechta,ROSTOCK SEAWOLVES,54:68
27,2021_22,1,Nürnberg Falcons BC,Artland Dragons,74:71
28,2021_22,1,VfL SparkassenStars Bochum,Medipolis SC Jena,74:91
29,2021_22,1,Eisbären Bremerhaven,RÖMERSTROM Gladiators Trier,84:87


In [70]:
table_reg_season[['pts+', 'pts-']] = table_reg_season.result.str.split(":", expand=True)
table_reg_season.head()

,season,matchday,home,guest,result,pts+,pts-
25,2021_22,1,spielfrei,Itzehoe Eagles,--:--,--,--
26,2021_22,1,RASTA Vechta,ROSTOCK SEAWOLVES,54:68,54,68
27,2021_22,1,Nürnberg Falcons BC,Artland Dragons,74:71,74,71
28,2021_22,1,VfL SparkassenStars Bochum,Medipolis SC Jena,74:91,74,91
29,2021_22,1,Eisbären Bremerhaven,RÖMERSTROM Gladiators Trier,84:87,84,87
